In [3]:
# contient le tokenizer et le modèle de sentiment
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# sert à faire les calculs du modèle
import torch
from elasticsearch import Elasticsearch
import pandas as pd

In [4]:
model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def predict_sentiment(texts):
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    scores, predictions = torch.max(probabilities, dim=-1)

    sentiment_map = {
        0: "Très négatif",
        1: "Négatif",
        2: "Neutre",
        3: "Positif",
        4: "Très positif"
    }

    return [
        (sentiment_map[p.item()], round(s.item() * 100, 2))
        for p, s in zip(predictions, scores)
    ]

In [5]:
try:

    es = Elasticsearch("http://localhost:9200")
    response = es.search(
        index="reviews",
        query={
            "exists": {
                "field": "user_review"
            }
        },
        size=200
    )

    reviews = []
    for hit in response["hits"]["hits"]:
        reviews.append(hit["_source"]["user_review"])
except Exception as e:
    print(f"Erreur Elasticsearch: {e}")

In [6]:
predictions = predict_sentiment(reviews)

data = []
for review, (sentiment, score) in zip(reviews, predictions):
    data.append({
        "user_review": review,
        "sentiment_prédit": sentiment,
        "confiance (%)": score
    })

df = pd.DataFrame(data)

df.head(50)

,user_review,sentiment_prédit,confiance (%)
0,Showroomprivé est un super site où l'on fait t...,Positif,53.00
1,"Je fais confiance à showroom,je n'ai jamais ét...",Positif,67.97
2,Toujours satisfaite de mes achats bon rapport ...,Positif,89.89
3,Tout est parfait malgré un délai un peu long,Négatif,47.66
4,très satisfait de show room prive,Positif,92.48
5,"Bonjour je déconseille fortement ce site, J'at...",Très négatif,75.24
6,"Délais respectés, facilité de la commande. je ...",Positif,45.26
7,très mauvaise expérience,Très négatif,65.61
8,Je fais suite à mon premier commentaire et vot...,Neutre,93.58
9,Facilité de récupération,Neutre,86.64
